# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [3]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import sys
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
# Path for scraper (run from repo root or community-contributions/asket)
for _path in ("week1", os.path.join(os.getcwd(), "..", "..", "week1")):
    _p = os.path.abspath(_path)
    if _p not in sys.path and os.path.isdir(_p):
        sys.path.insert(0, _p)
        break
from scraper import fetch_website_links, fetch_website_contents
from openai import OpenAI

In [4]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENROUTER_API_KEY')

if api_key and api_key.startswith('sk-or-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your OPENROUTER_API_KEY? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-5-nano'
openai = OpenAI(base_url="https://openrouter.ai/api/v1", api_key=api_key)

API key looks good so far


In [6]:
links = fetch_website_links("https://klingbo.com")
links

['/',
 '#how-it-works',
 '/contact',
 'mailto:hello@klingbo.com',
 'tel:+233531976985',
 '/',
 'https://auth.klingbo.com/login',
 'https://auth.klingbo.com/signup',
 '/blog',
 '/#faq',
 '/privacy',
 '/terms',
 '/status',
 '/careers',
 'mailto:hello@klingbo.com',
 '/contact?tab=feedback',
 '/ambassador',
 '/#features',
 '/docs',
 '/#features',
 'https://teachers.klingbo.com/auth',
 'https://teachers.klingbo.com/',
 '/about#mission',
 '/about#principles',
 '/for-parents',
 '/workforce',
 '/#features',
 '/#features',
 '/#features',
 '/#features',
 '/#features',
 '/#features',
 '/#features',
 '/#features',
 '/#features',
 '/#features',
 '/#features',
 '/#features',
 '/wassce',
 '/#features',
 '/#features',
 '/#features']

## First step: Have GPT-5-nano figure out which links are relevant

### Use a call to gpt-5-nano to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [7]:
link_system_prompt = """
You are provided with a list of links found on a webpage.
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [8]:
def get_links_user_prompt(url):
    user_prompt = f"""
Here is the list of links on the website {url} -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

"""
    links = fetch_website_links(url)
    user_prompt += "\n".join(links)
    return user_prompt

In [9]:
print(get_links_user_prompt("https://klingbo.com"))


Here is the list of links on the website https://klingbo.com -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

/
#how-it-works
/contact
mailto:hello@klingbo.com
tel:+233531976985
/
https://auth.klingbo.com/login
https://auth.klingbo.com/signup
/blog
/#faq
/privacy
/terms
/status
/careers
mailto:hello@klingbo.com
/contact?tab=feedback
/ambassador
/#features
/docs
/#features
https://teachers.klingbo.com/auth
https://teachers.klingbo.com/
/about#mission
/about#principles
/for-parents
/workforce
/#features
/#features
/#features
/#features
/#features
/#features
/#features
/#features
/#features
/#features
/#features
/#features
/wassce
/#features
/#features
/#features


In [10]:
def select_relevant_links(url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    return links
    

In [11]:
select_relevant_links("https://klingbo.com")

{'links': [{'type': 'about page (mission section)',
   'url': 'https://klingbo.com/about#mission'},
  {'type': 'about page (principles section)',
   'url': 'https://klingbo.com/about#principles'},
  {'type': 'home page', 'url': 'https://klingbo.com/'},
  {'type': 'blog page', 'url': 'https://klingbo.com/blog'},
  {'type': 'careers page', 'url': 'https://klingbo.com/careers'},
  {'type': 'ambassador page', 'url': 'https://klingbo.com/ambassador'},
  {'type': 'for parents page', 'url': 'https://klingbo.com/for-parents'},
  {'type': 'workforce page', 'url': 'https://klingbo.com/workforce'},
  {'type': 'contact page', 'url': 'https://klingbo.com/contact'}]}

In [12]:
def select_relevant_links(url):
    print(f"Selecting relevant links for {url} by calling {MODEL}")
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    print(f"Found {len(links['links'])} relevant links")
    return links

In [13]:
select_relevant_links("https://klingbo.com")

Selecting relevant links for https://klingbo.com by calling gpt-5-nano
Found 7 relevant links


{'links': [{'type': 'about page', 'url': 'https://klingbo.com/about'},
  {'type': 'about page', 'url': 'https://klingbo.com/about#mission'},
  {'type': 'about page', 'url': 'https://klingbo.com/about#principles'},
  {'type': 'contact page', 'url': 'https://klingbo.com/contact'},
  {'type': 'careers page', 'url': 'https://klingbo.com/careers'},
  {'type': 'ambassador program', 'url': 'https://klingbo.com/ambassador'},
  {'type': 'workforce page', 'url': 'https://klingbo.com/workforce'}]}

In [14]:
select_relevant_links("https://klingbo.com")

Selecting relevant links for https://klingbo.com by calling gpt-5-nano
Found 3 relevant links


{'links': [{'type': 'about page', 'url': 'https://klingbo.com/about#mission'},
  {'type': 'about page', 'url': 'https://klingbo.com/about#principles'},
  {'type': 'careers page', 'url': 'https://klingbo.com/careers'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT-5-nano

In [15]:
def fetch_page_and_all_relevant_links(url):
    contents = fetch_website_contents(url)
    relevant_links = select_relevant_links(url)
    result = f"## Landing Page:\n\n{contents}\n## Relevant Links:\n"
    for link in relevant_links['links']:
        result += f"\n\n### Link: {link['type']}\n"
        result += fetch_website_contents(link["url"])
    return result

In [16]:
print(fetch_page_and_all_relevant_links("https://klingbo.com"))

Selecting relevant links for https://klingbo.com by calling gpt-5-nano
Found 8 relevant links
## Landing Page:

Klingbo Intelligence - AI-Powered Education Platform

Klingbo Intelligence
Our Features
News
Educators & Enterprise
Careers
Login
Try for free
Ghana's Most Advanced
AI-Powered Learning
Platform
For students and institutions. Personalized tutoring, AI Coach, and study plans—or bring Klingbo to your school with teachers.klingbo.com.
Get Started
Explore Klingbo
Web, iOS & Android • Students: app.klingbo.com • Institutions: teachers.klingbo.com
app.klingbo.com
Welcome back!
Continue your learning journey
Courses
0
Progress
0
%
Groups
0
AI Tutor Active
Available 24/7 • Offline Capable
Ask me anything about your courses!
Study Groups
Join peers
Offline Mode
Always available
9:41
Klingbo
AI Tutor
How can I help you with your studies today?
Can you explain calculus derivatives?
AI Tutor
A derivative measures how a function changes as its input changes. It's the slope of the tangent l

In [17]:
brochure_system_prompt = """
You are an assistant that analyzes the contents of several relevant pages from a company website
and creates a short brochure about the company for prospective customers, investors and recruits.
Respond in markdown without code blocks.
Include details of company culture, customers and careers/jobs if you have the information.
"""

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# brochure_system_prompt = """
# You are an assistant that analyzes the contents of several relevant pages from a company website
# and creates a short, humorous, entertaining, witty brochure about the company for prospective customers, investors and recruits.
# Respond in markdown without code blocks.
# Include details of company culture, customers and careers/jobs if you have the information.
# """


In [18]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"""
You are looking at a company called: {company_name}
Here are the contents of its landing page and other relevant pages;
use this information to build a short brochure of the company in markdown without code blocks.\n\n
"""
    user_prompt += fetch_page_and_all_relevant_links(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [19]:
get_brochure_user_prompt("Klingbo", "https://klingbo.com")

Selecting relevant links for https://klingbo.com by calling gpt-5-nano
Found 6 relevant links


"\nYou are looking at a company called: Klingbo\nHere are the contents of its landing page and other relevant pages;\nuse this information to build a short brochure of the company in markdown without code blocks.\n\n\n## Landing Page:\n\nKlingbo Intelligence - AI-Powered Education Platform\n\nKlingbo Intelligence\nOur Features\nNews\nEducators & Enterprise\nCareers\nLogin\nTry for free\nGhana's Most Advanced\nAI-Powered Learning\nPlatform\nFor students and institutions. Personalized tutoring, AI Coach, and study plans—or bring Klingbo to your school with teachers.klingbo.com.\nGet Started\nExplore Klingbo\nWeb, iOS & Android • Students: app.klingbo.com • Institutions: teachers.klingbo.com\napp.klingbo.com\nWelcome back!\nContinue your learning journey\nCourses\n0\nProgress\n0\n%\nGroups\n0\nAI Tutor Active\nAvailable 24/7 • Offline Capable\nAsk me anything about your courses!\nStudy Groups\nJoin peers\nOffline Mode\nAlways available\n9:41\nKlingbo\nAI Tutor\nHow can I help you with you

In [20]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [21]:
create_brochure("Klingbo", "https://klingbo.com")

Selecting relevant links for https://klingbo.com by calling gpt-5-nano
Found 11 relevant links


Klingbo Intelligence — AI-Powered Education Platform

Overview
Klingbo Intelligence is Ghana’s most advanced AI-powered learning platform designed for students and institutions. Built by educators for African education, Klingbo delivers offline-capable, personalized tutoring, document intelligence, and study coaching tailored to local programs. The result: expert-level support 24/7 at a fraction of traditional tutoring costs.

What Klingbo Does
- Provides 24/7 personalized AI tutoring for all university courses and WASSCE preparation
- Offers offline-capable learning so access isn’t limited by connectivity
- Analyzes documents and extracts key concepts to create smart study materials
- Supports social learning with study groups, discussion forums, and peer collaboration
- Helps with exam prep, revision, and assignment feedback
- Delivers content and coaching you can trust in Ghanaian curricula (GTEC-approved content)

For Students
- Designed specifically for Ghanaian students—from WASSCE candidates to university excellence
- 24/7 AI Learning Assistant for courses in Law, Engineering, Medicine, Business, and more
- AI Tutor explains concepts and provides practice examples (e.g., derivatives, calculus)
- AI Coach plans your week and helps you stay on track
- Document Intelligence: scan lecture notes and PDFs, automatic summarization, key concept extraction

For Institutions & Educators
- Bring Klingbo to your school with teachers.klingbo.com
- Educators & Enterprise: collaborate to provide scalable, affordable AI tutoring
- Access to offline-capable tools and AI-powered study coaching that aligns with local programs
- GTEC-approved content ensures quality and relevance

Key Features
- AI Tutor: Available 24/7, offline-capable, answers questions and explains concepts
- AI Learning Assistant: Tailored help for university courses and WASSCE preparation
- AI Coach: Plans your week and keeps you on track
- Document Intelligence: Scan notes/PDFs, summarize, and extract concepts
- Social Learning: Form study groups, peer collaboration tools, and discussion forums
- Progress & Groups: Track learning progress and join or form study groups
- Platform Access: Web, iOS, and Android (Students: app.klingbo.com; Institutions: teachers.klingbo.com)

Platforms & Access
- Web, iOS, Android
- Students: app.klingbo.com
- Institutions: teachers.klingbo.com
- Get Started, Try for Free

Culture & Commitment
- Built for Students, By Educators: Klingbo combines frontline teaching experience with AI innovation
- A vertical AI platform focused on African education, delivering affordable, accessible support
- Emphasis on offline functionality, local relevance, and scalable learning for Ghana

Careers
- Klingbo welcomes opportunities to join a mission-driven team transforming education in Ghana
- Explore roles on the Klingbo Careers page

Get Involved
- Explore Klingbo and learn more about partnerships, products, and opportunities
- Links: Home, News, Educators & Enterprise, Careers
- Ready to start? Get started for free and see how Klingbo can transform learning

Join the next wave of AI-powered education in Ghana with Klingbo — your personal study assistant, anytime, anywhere.

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [22]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)

In [23]:
stream_brochure("Klingbo", "https://klingbo.com")

Selecting relevant links for https://klingbo.com by calling gpt-5-nano
Found 7 relevant links


# Klingbo Intelligence – AI-Powered Education Platform

Ghana’s most advanced AI-powered learning platform designed for students and institutions. Klingbo delivers personalized tutoring, AI coaching, and study planning—with offline capability to transform learning across Ghana and beyond.

## About Klingbo
- Built for Students, By Educators: A vertical AI platform designed for African education, delivering personalized tutoring that understands your courses and works offline.
- Offline-first design: Tutoring and study tools work even when internet access is limited.
- Local focus, global potential: Tailored to Ghanaian programs (WASSCE prep to university-level excellence) with affordable, high-quality support.
- Content you can trust: Includes GTEC-approved content and advanced document intelligence features.

## Who we serve
- University students (Law, Engineering, Medicine, Business, etc.)
- WASSCE candidates and general Ghanaian learners
- Educational institutions and educators looking to augment teaching with AI-powered tools

## What makes Klingbo unique
- Ghana-focused, Africa-ready: Specifically designed to meet local curricula and study needs.
- 24/7 AI Tutoring: Personal, on-demand help for any course, anytime.
- AI Coach & weekly planning: Personalized study schedules to keep you on track.
- Document Intelligence: Scan lecture notes and PDFs, with automatic summarization and key concept extraction.
- Social Learning: Form study groups, collaborate with peers, participate in discussion forums, and share knowledge.
- Progress visibility: Track courses, groups, and AI tutor activity at a glance.

## Our Platform at a Glance
- Availability: Web, iOS, and Android
- Student access: app.klingbo.com
- Institution access: teachers.klingbo.com
- AI Tutor: 24/7 support to answer questions about your courses (example: explains calculus derivatives and provides concrete examples)
- AI Coach: Plans your week and guides study routines
- Courses & Groups: Manage progress, join peers, and collaborate
- Document Intelligence: Scan notes/PDFs, summarize, extract key concepts
- Content: Includes GTEC-approved material for trusted study resources
- Offline Mode: Always accessible, even offline

## For Students
- 24/7 AI tutoring for any university course
- Subject coverage: Law, Engineering, Medicine, Business, and more
- Exam prep, revision, and assignment help with feedback
- Social learning: Connect with peers, form study groups, and participate in forums
- AI-powered study planning to keep you on track

## For Educators & Institutions
- Bring Klingbo to your school or university via teachers.klingbo.com
- Enhance teaching with AI-assisted tutoring and content
- Scalable, cost-effective support that complements traditional tutoring

## Careers & Opportunities
- Klingbo maintains a Careers section for opportunities across education technology and AI for Africa. If you’re passionate about empowering learners with offline-ready AI tools, explore openings via the Careers page.

## Get started
- Try Klingbo for free
- Get Started / Explore Klingbo today
- Access points: Students (app.klingbo.com) and Institutions (teachers.klingbo.com)

If you’d like, I can tailor this into a printable one-page brochure or a short slide-ready version for investors, customers, or potential hires.

In [24]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("Klingbo", "https://klingbo.com")

Selecting relevant links for https://klingbo.com by calling gpt-5-nano
Found 8 relevant links


Klingbo Intelligence
AI-Powered Education Platform for Ghana and Africa

Overview
Klingbo Intelligence is a vertical AI platform designed for African education, built for Ghanaian students and institutions. It delivers offline-capable, personalized AI tutoring, document intelligence, and study coaching that aligns with local curricula (GTEC and WAEC) to help students excel—24/7, anytime, anywhere.

What makes Klingbo unique
- 100% offline-capable AI tutoring, plus online sync when connected
- Ghana-focused content aligned to GTEC/WAEC
- 24/7 AI Tutor and AI Coach that plans your week
- Document intelligence: scan notes and PDFs, with automatic summarization and key concept extraction
- Social learning: study groups, peer collaboration, and discussion forums
- Substantial cost savings vs traditional tutoring

Key features at a glance
- AI Tutor: Always-on tutoring for university courses and WASSCE prep
- AI Coach & weekly study plans: Personal guidance to stay on track
- Offline-capable platform: Learn without internet, save data
- GTEC-approved content: Ghanaian curriculum with local context
- Documents & notes: Scan, summarize, extract key ideas
- Group learning: Build study groups and collaborate
- Platforms: Web, iOS, and Android (students app: app.klingbo.com; institutions: teachers.klingbo.com)

For Students
- What you get: 24/7 personalized tutoring across university subjects and WASSCE preparation (Law, Engineering, Medicine, Business, etc.)
- Help with exams, revision, and assignments
- Ghana-specific context and examples to make concepts relevant
- Study assistant that understands your courses and adapts to your pace
- Learn anywhere, anytime—even offline

For Educators & Institutions
- Bring Klingbo to your school with teachers.klingbo.com
- Cost-effective solution: up to 75% cheaper than traditional tutors
- Content aligned with local curricula (GTEC/WAEC)
- 24/7 AI tutoring reduces scheduling and staffing bottlenecks
- Tools to support teachers and learners with document intelligence and assessment feedback

Curriculum, Content & Partners
- Ghana-focused curriculum alignment (GTEC). Real examples from Ghanaian context.
- WAEC-ready content and exam prep materials
- Document intelligence supports lecture notes and PDFs with automatic summarization and concept extraction
- Partner-ready to scale with schools and universities

Platform, Access & Experience
- Across Web, iOS & Android
- Student app: app.klingbo.com; Institution app: teachers.klingbo.com
- 100% offline-capable experience, with data savings and seamless offline learning
- Real-time progress tracking: courses, groups, and AI tutor activity
- Social learning features to connect with peers and form study groups

Economics & Beta Highlights
- 75% cheaper than traditional tutors (approx. GHS 40/month vs GHS 200–500/month)
- Affordable monthly subscription; free during beta launch
- 24/7 AI tutoring with no scheduling constraints
- Early beta launched with strong promises: better grades, cost savings, and offline reliability

Culture & Careers
- Built for Students, By Educators: Klingbo emphasizes educator-led development and student-centered design
- A mission-driven, accessible approach to education that works offline and in low-connectivity contexts
- Ongoing opportunities to join the team through Careers; focus on roles that advance African education through AI
- Values collaboration, practicality, and impact—aiming to transform learning in Ghana and beyond

Why prospective customers choose Klingbo
- Ghana-first, Africa-ready AI education platform tailored to local curricula and contexts
- Significant cost savings compared to private tutoring
- Reliable offline learning ensures continuity even with limited internet access
- 24/7 support and personalized study planning improve outcomes and reduce study friction
- Easy to deploy in schools and universities with scalable, educator-led design

Investor & Partner benefits
- Large, underserved market with growing demand for affordable, accessible, high-quality education
- Differentiation through offline-first AI, local curriculum alignment, and social learning features
- Potential for expandability across African markets with similar curricula and connectivity challenges
- Evidence of impact: beta momentum, improved grades, and reduced tutoring costs

Get started
- Students: Try Klingbo for free and start your personalized AI-powered study journey
- Institutions: Explore bringing Klingbo to your school via teachers.klingbo.com
- Learn more about our Ghana-focused approach and our commitment to accessible education

Contact and next steps
- Visit Klingbo’s educational platform pages to explore features, news, and careers
- Look for collaboration and partnership opportunities to enhance learning outcomes

Klingbo Intelligence — your Ghanaian study companion, designed to help you learn smarter, faster, and more affordably.

### Build a brochure for any company

Edit the two variables below and run the cell to generate a brochure (streaming). Uses the same pipeline: select relevant links → fetch pages → LLM writes brochure.

In [25]:
COMPANY_NAME = "Klingbo"
COMPANY_URL = "https://klingbo.com"

stream_brochure(COMPANY_NAME, COMPANY_URL)

Selecting relevant links for https://klingbo.com by calling gpt-5-nano
Found 8 relevant links


# Klingbo Intelligence — AI-Powered Education Platform

Ghana’s most advanced AI-powered learning platform designed to empower students and institutions with personalized tutoring, offline capabilities, and intelligent study coaching.

## About Klingbo
- Built for African education, with a strong focus on Ghanaian programs (WASSCE prep and university excellence).
- An offline-capable, vertical AI platform that provides expert-level tutoring 24/7 at a fraction of traditional tutoring costs.
- Combines AI tutoring, document intelligence, social learning, and a personalized AI coach to help learners excel.

## For Students

- University Students: 24/7 AI Learning Assistant for any course (Law, Engineering, Medicine, Business, etc.), plus advanced research help, exam prep, and revision support.
- WASSCE Candidates: Tailored guidance and practice aligned with local curricula.
- AI Tutor: Interactive, always-on support that can answer questions and explain concepts (e.g., calculus derivatives) and adapt to your coursework.
- Document Intelligence: Scan lecture notes and PDFs, automatic summarization, and key concept extraction; content is GTEC-approved.
- Social Learning: Connect with peers across Ghana, form study groups, participate in discussion forums, and share knowledge.
- AI Coach: Plans your week and creates personalized study paths to keep you on track.
- Offline Mode: Learn anywhere, anytime—even without internet connectivity.

## For Institutions & Educators

- Bring Klingbo to your school with teachers.klingbo.com: a scalable way to augment classroom learning and reduce tutoring costs.
- Content Alignment: GTEC-approved materials ensure relevance and quality for local programs.
- Collaborative Learning: Facilitate peer-to-peer study groups and knowledge sharing within your institution.

## Key Features at a Glance

- AI Tutor: 24/7 tutoring for all courses, offline capable, always available.
- Study Groups & Social Learning: Peer collaboration tools, discussion forums, and knowledge sharing across Ghana.
- AI Coach & Weekly Planning: Personal study plans that adapt to your goals.
- Document Intelligence: AI-powered analysis, extraction, and summarization of notes and PDFs.
- Exam Prep, Revision & Feedback: Support for university-level courses and standardized exam readiness.
- Cross-Platform Availability: Web, iOS, and Android; separate student and institution experiences.

## Why Klingbo Stands Out

- Ghanaian-first Approach: Specifically designed to address local curricula and learning contexts.
- Accessibility & Affordability: Offline tutoring reduces barriers to quality education.
- Comprehensive Education Toolkit: From AI tutoring and planning to document analysis and peer learning.
- Trusted Content: GTEC-approved materials and curriculum-aligned support.

## Customers & Use Cases

- University students seeking 24/7 tutoring and research assistance across disciplines.
- WASSCE candidates looking for exam-focused practice and guidance.
- Institutions and schools wanting to enhance learning outcomes with a scalable AI-powered solution.

## Careers & Opportunities

- Klingbo maintains a Careers page for opportunities to join the team.
- The platform emphasizes collaboration between educators and technologists to transform education.
- If you’re passionate about education in Africa and building cutting-edge AI tools, Klingbo invites you to explore roles when they’re listed.

## Get Started

- Try Klingbo for free and explore how AI-powered tutoring, document intelligence, and social learning can boost academic success.
- Learn more or contact us via the Klingbo site’s Getting Started options (Students app: app.klingbo.com; Institutions: teachers.klingbo.com).

---

Klingbo Intelligence invites students, educators, and investors to join a wave of accessible, locally relevant AI-powered education designed to empower Ghanaian learners and institutions today—or anytime, anywhere.

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../../assets/business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../../assets/important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../../assets/resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../../assets/thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>